# Amazon SageMaker Processing Job 


기계 학습 (ML) 프로세스는 몇 단계로 구성됩니다. 먼저, 다양한 ETL 작업으로 데이터를 수집 한 다음 data의 pre-processing, 전통적인 기법 또는 사전 knowledge를 이용하여 데이터의 feature화, 마지막으로 알고리즘을 이용한 ML 모델을 학습합니다.

Apache Spark와 같은 분산 데이터 처리 프레임 워크는 학습을 위해 dataset의 pre-processing하는데 사용합니다. 이 노트북에서는 Amazon SageMaker Processing에서 기본 설치된 Apache Spark의 기능을 활용하여 처리 워크로드를 실행합니다.

![](img/prepare_dataset_bert.png)

![](img/processing.jpg)


# Setup Environment


* 모델 학습에 사용되는 S3 bucket과 prefix 가 필요합니다.
* 학습과 processing을 위해 IAM role은 dataset에 액세스가 가능해야 합니다.

In [1]:
import sagemaker
from time import gmtime, strftime
import boto3

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

# Setup Input Data

In [2]:
# Inputs
s3_input_data = 's3://{}/amazon-reviews-pds/tsv/'.format(bucket)
print(s3_input_data)

s3://sagemaker-us-east-1-322537213286/amazon-reviews-pds/tsv/


In [3]:
!aws s3 ls $s3_input_data

2020-12-08 04:14:36   18997559 amazon_reviews_us_Digital_Software_v1_00.tsv.gz
2020-12-08 04:14:38   27442648 amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz
2020-12-08 04:14:41  193389086 amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz


# Processing Job을 수행할 Spark Docker Image

이 HOL에서는 `./container` 폴더 내에 Spark container 이미지를 포함합니다. container는 모든 Spark 구성의 부트스트랩을 처리하고 `spark-submit` CLI를 wrapper해서 제공합니다. 상위 레벨에서는,

* A set of default Spark/YARN/Hadoop configurations
* A bootstrapping script for configuring and starting up Spark master/worker nodes
* A wrapper around the `spark-submit` CLI to submit a Spark application

container 빌드와 push 절차가 완료된 후 dataset의 처리를 수행하는 관리형 분산 Spark 어플리케이션을 수행사는 것은 Amazon SageMaker Python SDK 사용합니다.

In [4]:
docker_repo = 'amazon-reviews-spark-processor'
docker_tag = 'latest'

In [21]:
!docker build -t $docker_repo:$docker_tag -f container/Dockerfile ./container

Sending build context to Docker daemon  4.385MB
Step 1/37 : FROM openjdk:8-jre-slim
 ---> 8c3c0e49c694
Step 2/37 : RUN apt-get update
 ---> Using cache
 ---> 4b70a64d75f7
Step 3/37 : RUN apt-get install -y curl unzip python3 python3-setuptools python3-pip python-dev python3-dev python-psutil
 ---> Using cache
 ---> 3c014c905e2f
Step 4/37 : RUN pip3 install py4j psutil==5.6.5 numpy==1.17.4
 ---> Using cache
 ---> bbbebd176ae5
Step 5/37 : RUN apt-get clean
 ---> Using cache
 ---> 1a537b0b84c5
Step 6/37 : RUN rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 57462dab1261
Step 7/37 : ENV PYTHONHASHSEED 0
 ---> Using cache
 ---> de525fa99d2e
Step 8/37 : ENV PYTHONIOENCODING UTF-8
 ---> Using cache
 ---> 8c0bf04bc4b6
Step 9/37 : ENV PIP_DISABLE_PIP_VERSION_CHECK 1
 ---> Using cache
 ---> 3e66672125fd
Step 10/37 : ENV HADOOP_VERSION 3.2.1
 ---> Using cache
 ---> 4b1ff6b5ab23
Step 11/37 : ENV HADOOP_HOME /usr/hadoop-$HADOOP_VERSION
 ---> Using cache
 ---> 6740c3f3450e
Step 12/37 : ENV HADOOP

Spark container의 Amazon Elastic Container Registry(Amazon ECR) 리포지토리를 생성하고 image를 push합니다.

In [22]:
import boto3
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name

image_uri = '{}.dkr.ecr.{}.amazonaws.com/{}:{}'.format(account_id, region, docker_repo, docker_tag)
print(image_uri)

322537213286.dkr.ecr.us-east-1.amazonaws.com/amazon-reviews-spark-processor:latest


### ECR repository 생성과 docker image를 push하기

In [23]:
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


### `RepositoryNotFoundException` 오류는 무시하셔도 됩니다. 즉시 repository를 생성합니다.

In [24]:
!aws ecr describe-repositories --repository-names $docker_repo || aws ecr create-repository --repository-name $docker_repo

{
    "repositories": [
        {
            "repositoryArn": "arn:aws:ecr:us-east-1:322537213286:repository/amazon-reviews-spark-processor",
            "registryId": "322537213286",
            "repositoryName": "amazon-reviews-spark-processor",
            "repositoryUri": "322537213286.dkr.ecr.us-east-1.amazonaws.com/amazon-reviews-spark-processor",
            "createdAt": 1596262176.0,
            "imageTagMutability": "MUTABLE",
            "imageScanningConfiguration": {
                "scanOnPush": false
            },
            "encryptionConfiguration": {
                "encryptionType": "AES256"
            }
        }
    ]
}


In [25]:
!docker tag $docker_repo:$docker_tag $image_uri

In [26]:
!docker push $image_uri

The push refers to repository [322537213286.dkr.ecr.us-east-1.amazonaws.com/amazon-reviews-spark-processor]

eaa0235e: Preparing 
edaba6e6: Preparing 
e5ebb758: Preparing 
4ee3cec3: Preparing 
5a78eb75: Preparing 
80f9b7a3: Preparing 
ce9863fb: Preparing 
865a3806: Preparing 
4961b07e: Preparing 
24123e10: Preparing 
0f9685ba: Preparing 
0ae3ee2c: Preparing 
f6d86fae: Preparing 
a741a80e: Preparing 
7975ff2a: Preparing 
9ffee0cb: Preparing 
5206ae8c: Preparing 
98b79cde: Preparing 
5206ae8c: Layer already exists latest: digest: sha256:e105201545d54b91696c127bfb809ad88c5339e7e54a5864863f532806322114 size: 4318


# Amazon SageMaker Processing Jobs 으로 Job 수행

Amazon SageMaker Python SDK를 사용하여 Processing job을 실행합니다. Spark container와 job configuration에서 processing에 대한 Spark ML script를 사용합니다.

In [27]:
!pygmentize src_dir/preprocess-spark-text-to-bert.py

from __future__ import print_function
from __future__ import unicode_literals

import time
import sys
import os
import shutil
import csv
import collections
import subprocess
import sys
#subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'pip', '--upgrade'])
#subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'wrapt', '--upgrade', '--ignore-installed'])
#subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'tensorflow==2.1.0', '--ignore-installed'])
import tensorflow as tf
print(tf.__version__)
#subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'transformers==2.8.0'])
from transformers import DistilBertTokenizer

import pyspark
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.functions import *
from pyspark.ml.linalg import DenseVector
from pyspark.sql.functions import split
from pyspark.sql.functions import udf, col
from pyspark.sql.types import *

tokenizer = DistilBertTokenizer.from_pretrained('distil

    ])

    bert_transformer = udf(lambda text, label: convert_input(text, label), tfrecord_schema)

    spark.udf.register('bert_transformer', bert_transformer)

    transformed_df = features_df.select(bert_transformer('star_rating', 'review_body').alias('tfrecords'))
    transformed_df.show(truncate=False)

    flattened_df = transformed_df.select('tfrecords.*')
    flattened_df.show()

    # Split 90-5-5%
    train_df, validation_df, test_df = flattened_df.randomSplit([0.9, 0.05, 0.05])

    train_df.write.format('tfrecord').option('recordType', 'Example').save(path=s3_output_train_data)
    print('Wrote to output file:  {}'.format(s3_output_train_data))
    
    validation_df.write.format('tfrecord').option('recordType', 'Example').save(path=s3_output_validation_data)
    print('Wrote to output file:  {}'.format(s3_output_validation_data))

    test_df.write.format('tfrecord').option('recordType', 'Example').save(path=s3_output_test_data)    
    print('Wrote to output file:  {}'.f

In [28]:
from sagemaker.processing import ScriptProcessor

processor = ScriptProcessor(base_job_name='spark-amazon-reviews-processor',
                            image_uri=image_uri,
                            command=['/opt/program/submit'],
                            role=role,
                            instance_count=2, # instance_count needs to be > 1 or you will see the following error:  "INFO yarn.Client: Application report for application_ (state: ACCEPTED)"
                            instance_type='ml.r5.xlarge',
                            env={'mode': 'python'})

# Setup Output Data

In [29]:
from time import gmtime, strftime
timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

output_prefix = 'amazon-reviews-spark-processor-{}'.format(timestamp_prefix)

In [30]:
train_data_bert_output = 's3://{}/{}/output/bert-train'.format(bucket, output_prefix)
validation_data_bert_output = 's3://{}/{}/output/bert-validation'.format(bucket, output_prefix)
test_data_bert_output = 's3://{}/{}/output/bert-test'.format(bucket, output_prefix)

print(train_data_bert_output)
print(validation_data_bert_output)
print(test_data_bert_output)

s3://sagemaker-us-east-1-322537213286/amazon-reviews-spark-processor-2020-12-08-14-18-08/output/bert-train
s3://sagemaker-us-east-1-322537213286/amazon-reviews-spark-processor-2020-12-08-14-18-08/output/bert-validation
s3://sagemaker-us-east-1-322537213286/amazon-reviews-spark-processor-2020-12-08-14-18-08/output/bert-test


In [31]:
from sagemaker.processing import ProcessingOutput

processor.run(code='./src_dir/preprocess-spark-text-to-bert.py',
              arguments=['s3_input_data', s3_input_data,
                         's3_output_train_data', train_data_bert_output,
                         's3_output_validation_data', validation_data_bert_output,
                         's3_output_test_data', test_data_bert_output,                         
              ],
              # We need this dummy output to allow us to call 
              #    ProcessingJob.from_processing_name() later 
              #    to describe the job and poll for Completed status
              outputs=[
                       ProcessingOutput(s3_upload_mode='EndOfJob',
                                        output_name='dummy-output',
                                        source='/opt/ml/processing/output')
              ],          
              logs=True,
              wait=False
)


Job Name:  spark-amazon-reviews-processor-2020-12-08-14-18-08-603
Inputs:  [{'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-322537213286/spark-amazon-reviews-processor-2020-12-08-14-18-08-603/input/code/preprocess-spark-text-to-bert.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'dummy-output', 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-322537213286/spark-amazon-reviews-processor-2020-12-08-14-18-08-603/output/dummy-output', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]


In [32]:
from IPython.core.display import display, HTML

spark_processing_job_name = processor.jobs[-1].describe()['ProcessingJobName']

display(HTML('<b>Review <a href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(region, spark_processing_job_name)))


In [33]:
from IPython.core.display import display, HTML

# This is different than the job name because we are not using ProcessingOutput's in this Spark ML case.
spark_processing_job_s3_output_prefix = output_prefix

display(HTML('<b>Review <a href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Spark Job Has Completed</b>'.format(bucket, spark_processing_job_s3_output_prefix, region)))


# List Processing Jobs through boto3 Python SDK

In [34]:
import boto3

client = boto3.client('sagemaker')
client.list_processing_jobs()

{'ProcessingJobSummaries': [{'ProcessingJobName': 'spark-amazon-reviews-processor-2020-12-08-14-18-08-603',
   'ProcessingJobArn': 'arn:aws:sagemaker:us-east-1:322537213286:processing-job/spark-amazon-reviews-processor-2020-12-08-14-18-08-603',
   'CreationTime': datetime.datetime(2020, 12, 8, 14, 18, 9, 6000, tzinfo=tzlocal()),
   'LastModifiedTime': datetime.datetime(2020, 12, 8, 14, 18, 9, 6000, tzinfo=tzlocal()),
   'ProcessingJobStatus': 'InProgress'},
  {'ProcessingJobName': 'tensorflow-training-2020-1-Overtraining-fd755f53',
   'ProcessingJobArn': 'arn:aws:sagemaker:us-east-1:322537213286:processing-job/tensorflow-training-2020-1-overtraining-fd755f53',
   'CreationTime': datetime.datetime(2020, 12, 8, 12, 55, 21, 768000, tzinfo=tzlocal()),
   'ProcessingEndTime': datetime.datetime(2020, 12, 8, 13, 3, 21, 336000, tzinfo=tzlocal()),
   'LastModifiedTime': datetime.datetime(2020, 12, 8, 13, 3, 21, 340000, tzinfo=tzlocal()),
   'ProcessingJobStatus': 'Completed'},
  {'ProcessingJob

# Please Wait Until the Processing Job Completes
Re-run this next cell until the job status shows `Completed`.

In [35]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(processing_job_name=spark_processing_job_name,
                                                                            sagemaker_session=sagemaker_session)

processing_job_description = running_processor.describe()

processing_job_status = processing_job_description['ProcessingJobStatus']
print('\n')
print(processing_job_status)
print('\n')

print(processing_job_description)



InProgress


{'ProcessingInputs': [{'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-322537213286/spark-amazon-reviews-processor-2020-12-08-14-18-08-603/input/code/preprocess-spark-text-to-bert.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}], 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'dummy-output', 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-322537213286/spark-amazon-reviews-processor-2020-12-08-14-18-08-603/output/dummy-output', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}, 'AppManaged': False}]}, 'ProcessingJobName': 'spark-amazon-reviews-processor-2020-12-08-14-18-08-603', 'ProcessingResources': {'ClusterConfig': {'InstanceCount': 2, 'InstanceType': 'ml.r5.xlarge', 'VolumeSizeInGB': 30}}, 'StoppingCondition': {'MaxRuntimeInSeconds': 86400}, 'AppSpecification': {'ImageUri': '

In [ ]:
running_processor.wait()

2020-12-08 14:22:09,259 INFO namenode.NameNode: STARTUP_MSG: 
/************************************************************
STARTUP_MSG: Starting NameNode
STARTUP_MSG:   host = algo-1/10.0.238.61
STARTUP_MSG:   args = [-format, -force]
STARTUP_MSG:   version = 3.2.1
STARTUP_MSG:   classpath = /usr/hadoop-3.2.1/etc/hadoop:/usr/hadoop-3.2.1/share/hadoop/common/lib/commons-cli-1.2.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/jetty-util-9.3.24.v20180605.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/json-smart-2.3.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/curator-recipes-2.13.0.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/stax2-api-3.1.4.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/nimbus-jose-jwt-4.41.1.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/paranamer-2.3.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/hadoop-annotations-3.2.1.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/jetty-servlet-9.3.24.v20180605.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/checker-qual-2.5.2.jar:/usr/ha

Starting resourcemanager
Starting nodemanagers
localhost: /usr/hadoop-3.2.1/bin/../libexec/hadoop-functions.sh: line 982: ssh: command not found
2020-12-08 14:22:22,842 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2020-12-08 14:22:23.695715: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-12-08 14:22:23.695809: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-12-08 14:22:23.695820: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are in

2020-12-08 14:22:27,865 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2020-12-08 14:22:28,623 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2020-12-08 14:22:28,864 INFO yarn.Client: Uploading resource file:/usr/spark-2.4.6/python/lib/pyspark.zip -> hdfs://10.0.238.61/user/root/.sparkStaging/application_1607437341230_0001/pyspark.zip
2020-12-08 14:22:28,873 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2020-12-08 14:22:28,895 INFO yarn.Client: Uploading resource file:/usr/spark-2.4.6/python/lib/py4j-0.10.7-src.zip -> hdfs://10.0.238.61/user/root/.sparkStaging/application_1607437341230_0001/py4j-0.10.7-src.zip
2020-12-08 14:22:28,902 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2020-12-08 14:22:29,050 INFO yarn.Clie

2020-12-08 14:23:01,267 INFO scheduler.TaskSetManager: Finished task 0.0 in stage 0.0 (TID 0) in 2351 ms on algo-2 (executor 1) (1/1)
2020-12-08 14:23:01,270 INFO cluster.YarnScheduler: Removed TaskSet 0.0, whose tasks have all completed, from pool 
2020-12-08 14:23:01,274 INFO scheduler.DAGScheduler: ResultStage 0 (showString at NativeMethodAccessorImpl.java:0) finished in 2.434 s
2020-12-08 14:23:01,277 INFO scheduler.DAGScheduler: Job 0 finished: showString at NativeMethodAccessorImpl.java:0, took 2.469554 s
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+------------

2020-12-08 14:23:07,204 INFO scheduler.TaskSetManager: Finished task 0.0 in stage 4.0 (TID 4) in 2324 ms on algo-2 (executor 1) (1/1)
2020-12-08 14:23:07,204 INFO cluster.YarnScheduler: Removed TaskSet 4.0, whose tasks have all completed, from pool 
2020-12-08 14:23:07,205 INFO python.PythonAccumulatorV2: Connected to AccumulatorServer at host: 127.0.0.1 port: 53275
2020-12-08 14:23:07,206 INFO scheduler.DAGScheduler: ResultStage 4 (showString at NativeMethodAccessorImpl.java:0) finished in 2.342 s
2020-12-08 14:23:07,206 INFO scheduler.DAGScheduler: Job 4 finished: showString at NativeMethodAccessorImpl.java:0, took 2.344743 s
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

2020-12-08 14:46:02,183 INFO scheduler.TaskSetManager: Finished task 1.0 in stage 6.0 (TID 7) in 1371753 ms on algo-2 (executor 1) (1/2)
2020-12-08 14:52:36,543 INFO spark.ContextCleaner: Cleaned accumulator 114
2020-12-08 14:52:36,547 INFO storage.BlockManagerInfo: Removed broadcast_7_piece0 on 10.0.238.61:36471 in memory (size: 6.4 KB, free: 364.8 MB)
2020-12-08 14:52:36,549 INFO storage.BlockManagerInfo: Removed broadcast_7_piece0 on algo-2:35201 in memory (size: 6.4 KB, free: 11.9 GB)
2020-12-08 14:52:36,556 INFO spark.ContextCleaner: Cleaned accumulator 160
2020-12-08 14:52:36,557 INFO spark.ContextCleaner: Cleaned accumulator 177
2020-12-08 14:52:36,557 INFO spark.ContextCleaner: Cleaned accumulator 178
2020-12-08 14:52:36,557 INFO spark.ContextCleaner: Cleaned accumulator 174
2020-12-08 14:52:36,557 INFO spark.ContextCleaner: Cleaned accumulator 179
2020-12-08 14:52:36,557 INFO spark.ContextCleaner: Cleaned accumulator 165
2020-12-08 14:52:36,557 INFO spark.ContextCleaner: Clean

2020-12-08 15:52:36,569 INFO spark.ContextCleaner: Cleaned accumulator 200
2020-12-08 15:52:36,569 INFO spark.ContextCleaner: Cleaned accumulator 224
2020-12-08 15:52:36,569 INFO spark.ContextCleaner: Cleaned accumulator 206
2020-12-08 15:52:36,569 INFO spark.ContextCleaner: Cleaned accumulator 203
2020-12-08 15:52:36,569 INFO spark.ContextCleaner: Cleaned accumulator 221
2020-12-08 15:52:36,569 INFO spark.ContextCleaner: Cleaned accumulator 208
2020-12-08 15:52:36,569 INFO spark.ContextCleaner: Cleaned accumulator 218
2020-12-08 15:52:36,569 INFO spark.ContextCleaner: Cleaned accumulator 210
2020-12-08 15:52:36,569 INFO spark.ContextCleaner: Cleaned accumulator 223
2020-12-08 15:52:36,569 INFO spark.ContextCleaner: Cleaned accumulator 205
2020-12-08 15:52:36,569 INFO spark.ContextCleaner: Cleaned accumulator 219
2020-12-08 15:52:36,569 INFO spark.ContextCleaner: Cleaned accumulator 209
2020-12-08 15:52:36,569 INFO spark.ContextCleaner: Cleaned accumulator 189
2020-12-08 15:52:36,569 I

<h2><span style="color:red">위 Processing Job이 완료되기 전까지 기다려 주시기 바랍니다.</span></h2>


# the Processed Output Dataset 확인

In [ ]:
!aws s3 ls --recursive $train_data_bert_output/

In [ ]:
!aws s3 ls --recursive $validation_data_bert_output/

In [ ]:
!aws s3 ls --recursive $test_data_bert_output/

In [ ]:
train_data = './data-tfrecord/bert-train'
validation_data = './data-tfrecord/bert-validation'
test_data = './data-tfrecord/bert-test'

!aws s3 cp $train_data_bert_output $train_data --recursive
!aws s3 cp $validation_data_bert_output $validation_data --recursive
!aws s3 cp $test_data_bert_output $test_data --recursive

In [ ]:
%store train_data_bert_output train_data

In [ ]:
%store validation_data_bert_output validation_data

In [ ]:
%store test_data_bert_output test_data